In [1]:
from multiprocessing.pool import ThreadPool
from multiprocessing import cpu_count
from glob import glob
import os


def get_frame(checkpoint_dir, input_image_path, dir_of_frames, cpu_count=1): # use only 1 CPU to generate 1 frame
    output_image_path = os.path.join(dir_of_frames, "{}{}".format(checkpoint_dir.split('_')[-1], os.path.splitext(input_image_path)[-1]))    
    os.system("python evaluate_upgraded.py --checkpoint {} --in-path {} --out-path {} --device /cpu:{}".format(
        checkpoint_dir, input_image_path, output_image_path, cpu_count))

def get_frame_wrapper(args):
    get_frame(*args)

def create_progress_frames(parent_dir_of_checkpoints, input_image_path, output_dir_of_frames):
    checkpoint_dirs = (path for path in glob(os.path.join(parent_dir_of_checkpoints, '*')) 
                       if os.path.isdir(path) and 'checkpoint' in path)
    checkpoint_dirs = sorted(checkpoint_dirs, key=lambda path: int(path.split('_')[-1]))
    
    args_list = zip(checkpoint_dirs, [input_image_path] * len(checkpoint_dirs), [output_dir_of_frames] * len(checkpoint_dirs))
    pool = ThreadPool(cpu_count())
    temp = pool.map(get_frame_wrapper, args_list)
    pool.close()
    del pool

In [2]:
%%time
create_progress_frames('test_dir_small_data', 'examples/style/udnie.jpg', 'saved_frames')

CPU times: user 96 ms, sys: 84 ms, total: 180 ms
Wall time: 2min 7s


In [9]:
import imageio
from glob import glob
import os
import math
import numpy as np


def create_video_from_frames(input_dir_of_frames, output_video_path, video_duration_in_seconds, fps):
    sorted_frame_names = sorted(glob(os.path.join(output_dir_of_frames, '*')), key=lambda filename: 
                                int(os.path.splitext(os.path.basename(filename))[0]))
    num_repeats = math.ceil(video_duration_in_seconds * fps / len(sorted_frame_names))
    repeated_frame_indices = np.repeat(range(len(sorted_frame_names)), num_repeats)
    mask_sample = np.random.choice(repeated_frame_indices, video_duration_in_seconds * fps, replace=False)
    mask_sample.sort()
    frame_paths_for_video = np.array(sorted_frame_names)[mask_sample]
    imageio.mimsave(output_video_path, [imageio.imread(frame_path) for frame_path in frame_paths_for_video], fps=fps)

In [12]:
create_video_from_frames(input_dir_of_frames='saved_frames/', output_video_path='temp_movie.mp4',
                        video_duration_in_seconds=15, fps=16) ### hard coded parameters

In [18]:
%%HTML
<video controls>
  <source src="temp_movie.mp4" type="video/mp4">
</video>

In [1]:
!mkdir test_dir_small_data_2

mkdir: cannot create directory ‘test_dir_small_data_2’: File exists


In [ ]:
import style_with_multiple_checkpoints


style_with_multiple_checkpoints.create_checkpoints(
        # mandatory arguments
        style='examples/style/rain_princess.jpg', checkpoint_dir='test_dir_small_data_2/', 
        device_and_number='/gpu:0',
        # useful argments
        test='examples/content/stata.jpg', test_dir='test_dir_small_data_2/', 
        epochs=20, checkpoint_iterations=1, batch_size=20, 
        max_runtime_in_minutes=1, log_file='test_dir_small_data_2/runtime.txt',
        # usually don't want to tamper with this argument
        train_path="data/train2014_small/"
)


import style_with_multiple_checkpoints
style_with_multiple_checkpoints.create_checkpoints(
        # mandatory arguments
        style='examples/style/rain_princess.jpg', checkpoint_dir='test_dir_small_data_2/', 
        device_and_number='/cpu:0', # use all CPUs
        # useful argments
        test='examples/content/stata.jpg', test_dir='test_dir_small_data_2/', 
        epochs=20, checkpoint_iterations=1, batch_size=20,
        max_runtime_in_minutes=1, log_file='test_dir_small_data_2/runtime.txt',
        # usually don't want to tamper with this argument
        train_path="data/train2014_small/"
)


python style_with_multiple_checkpoints.py --style examples/style/rain_princess.jpg --checkpoint-dir test_dir_small_data_2/ \
  --device /gpu:0 --test examples/content/stata.jpg --test-dir test_dir_small_data_2/ --epochs 20 \
  --checkpoint-iterations 1 --batch-size 20 --max-runtime-in-minutes 1 --train-path data/train2014_small/ \
  &> test_dir_small_data_2/runtime.txt
# identical to 
python style_with_multiple_checkpoints.py --style examples/style/rain_princess.jpg --checkpoint-dir test_dir_small_data_2/ \
  --device /gpu:0 --test examples/content/stata.jpg --test-dir test_dir_small_data_2/ --epochs 20 \
  --checkpoint-iterations 1 --batch-size 20 --max-runtime-in-minutes 1 --train-path data/train2014_small/ \
  --log_file test_dir_small_data_2/runtime.txt --log_file test_dir_small_data_2/runtime.txt

In [ ]:
runtime to infinity None 
logfile None done
CPU mode: done

glances

In [ ]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()